In [2]:
import xarray as xr
import numpy as np
import glob
import os.path

from dask.distributed import Client
from dask.diagnostics import ProgressBar

from aggfly import dataset, regions, grid_weights
from aggfly.aggregate import TemporalAggregator, SpatialAggregator, get_time_dim

ProgressBar().register()

In [3]:
# Set file output name/path
output_path = "/home3/dth2133/data/aggregated/counties/"
output_name = "usa_counties_corn_monthly"
output_varn = "ddrev29"
csv = True

In [4]:
# Open shapefile containing region features.
georegions = regions.from_name('counties')

# Open example climate dataset to calculate grid weights.
clim = dataset.from_path(
    f"/home3/dth2133/data/usa/usa-t2m_tempPrecLand2019.zarr", 
    't2m', 
    'zarr', 
    preprocess=dataset.timefix_era5l)

# Clip climate data to the US (raw data are global)
clim.clip_data_to_georegions_extent(georegions)

# Rechunk dataset to optimize multithreading
clim.rechunk((5, 578, -1, -1, -1, -1))

# Calculate area and crop layer weights.
weights = grid_weights.from_objects(clim, georegions, crop='corn')

# This object aggregates cells within a region to the average across 
# cells, weighted by `weights`, which in this case are the area of the
# cell and the share of the cell with corn crops.
spatial = SpatialAggregator('avg')


# This object covers aggregating hourly and daily data to the yearly 
# level
daily = TemporalAggregator(
    'time', 
    agg_from='hour',
    agg_to='day',
    ddargs=[29,999,0])


annual = TemporalAggregator(
    'sum', 
    agg_from='day',
    agg_to='month')



In [5]:
# Calculate the grid weights
# w = weights.weights()
# w.to_netcdf('/home3/dth2133/data/test_weights.nc4')
w = xr.open_dataset('/home3/dth2133/data/test_weights.nc4', chunks='auto')
# w
w = w['multiply-318570b5896b046a3daaa85e867883cc']
# w.data
w.data = w.data.rechunk(-1)

In [6]:
clim.rechunk(-1)

In [19]:
from aggfly.aggregate import nb_expander

In [25]:
yl, xl = np.nonzero(test[0,:,:])

In [27]:
xl

array([356, 357, 358, 355, 356, 357, 358, 355, 356, 357, 358, 359, 356,
       357, 358, 359])

In [6]:
# clim.rechunk((-1,-1,1,1,1,1))
# clim.da

,Array,Chunk
Bytes,4.81 GiB,564.45 kiB
Shape,"(250, 578, 1, 12, 31, 24)","(250, 578, 1, 1, 1, 1)"
Count,32150 Tasks,8928 Chunks
Type,float32,numpy.ndarray


In [7]:
t = spatial.map_execute(clim, w, update=False)

In [9]:
# clim.da
o = t.compute()

[########################################] | 100% Completed | 39.4s
[########################################] | 100% Completed | 39.5s


In [35]:
import numba
from numba import prange
@numba.njit(fastmath=True, parallel=True)
def avg(frame, weight, poly):

    frame_shp = frame.shape
    frame = nb_expander(frame)
    res = np.zeros((weight.shape[0],) + frame.shape[2:], dtype=np.float64)
    wes = np.zeros((weight.shape[0],) + frame.shape[2:], dtype=np.float64)
    for r in prange(weight.shape[0]):
        # print(r)
        yl, xl = np.nonzero(weight[r,:,:])
        for a in prange(frame.shape[2]):
            for m in prange(frame.shape[3]):
                for d in prange(frame.shape[4]):
                    for h in prange(frame.shape[5]):
                        for l in prange(len(yl)):
                            y = yl[l]
                            x = xl[l]
                            # I can't believe this was actually the solution
                            # https://github.com/numba/numba/issues/2919
                            if int(frame[y,x,a,m,d,h]) != -9223372036854775808:
                                f = frame[y,x,a,m,d,h]
                                w = weight[r,y,x]
                                res[r,a,m,d,h] += f * w
                                wes[r,a,m,d,h] += w
    out = res/wes
    return out.reshape((weight.shape[0],) + frame_shp[2:])

In [36]:
%%time
o = avg(c.data, wc.data, 1)

CPU times: user 45.5 s, sys: 6.71 s, total: 52.2 s
Wall time: 5.14 s


In [33]:
o

array([[[[[ 1.18437187e+01,  1.11362726e+01,  1.02311648e+01, ...,
            7.22834670e+00,  6.85088121e+00,  6.38505034e+00],
          [ 5.99126697e+00,  5.58371132e+00,  5.21436993e+00, ...,
            5.19990552e+00,  4.95414357e+00,  4.55256743e+00],
          [ 4.22389061e+00,  3.96178311e+00,  3.71546996e+00, ...,
            7.53575929e+00,  7.23518757e+00,  6.34117546e+00],
          ...,
          [ 3.53613711e+00,  1.14951100e+00, -4.52348209e-01, ...,
            1.04236269e-01,  4.04382493e-01, -6.72213884e-01],
          [-2.02137497e+00, -2.47880421e+00, -2.65317960e+00, ...,
           -7.69634630e+00, -7.70127486e+00, -8.04613725e+00],
          [-8.26656297e+00, -8.15265394e+00, -8.15094651e+00, ...,
            2.53724081e+00,  2.54123929e+00,  1.24592858e+00]],

         [[-5.17088341e-01, -4.35227247e-01, -1.48681461e-01, ...,
            9.37138710e+00,  8.85893052e+00,  7.36846118e+00],
          [ 5.32631977e+00,  3.90569609e+00,  2.71315664e+00, ...,
      

In [13]:
def aggregate_era5l_t2m(path):
    # path = "/home3/dth2133/data/usa/usa-t2m_tempPrecLand1969.zarr"
    # Open climate dataset.
    clim = dataset.from_path(
        path, 
        't2m', 
        'zarr', 
        preprocess=dataset.timefix_era5l) # Kelvin to Celsius

    # Clip climate data to the US (raw data are global)
    clim.clip_data_to_georegions_extent(georegions)
    # Rechunk dataset to optimize multithreading
    clim.rechunk((5, 578, -1, -1, -1))

    # Update climate dataset in `clim` to one collapsed over
    # hour and day based upon `temporal` definition above.
    daily.map_execute(clim)
    annual.map_execute(clim)
    clim.rechunk(-1)

    # Again update, but with (weighted) spatial collapse over regions.
    spatial.map_execute(clim, w)

    # Send back the aggregated climate data.
    return clim.da


In [14]:
# Check years from the input path - this just creates a vector of years for
# which my raw climate data are available, e.g. [1970, 1971, ...]
import numpy as np
import glob
from os.path import basename
files = np.sort([x for x in glob.glob('/home3/dth2133/data/usa/*t2m_*')])
# Loop over years and aggregate.
output = list()
for f in files:
    print(f)
    output.append(aggregate_era5l_t2m(f))

/home3/dth2133/data/usa/usa-t2m_tempPrecLand1951.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1952.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1953.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1954.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1955.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1956.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1957.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1958.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1960.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1961.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1962.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1963.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1964.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1965.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1966.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1967.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1968.zarr
/home3/dth2133/data/usa/usa-t2m_tempPrecLand1969.zarr
/home3/dth2133/data/usa/usa-

In [15]:
da = xr.concat(output, dim='year').compute()

[########################################] | 100% Completed | 26min 16.7s


In [21]:
#  Convert to dataset format
ds = da.to_dataset(name=output_varn)
#  Append to existing array of results
ds.to_zarr(os.path.join(output_path, output_name+'.zarr'), mode='a') 

In [22]:
# Save to csv for Stata
if csv:
    ds = xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))
    ds.to_dataframe().to_csv(os.path.join(output_path, output_name+'.csv'))

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100